A script to transform a json file which is composed by a list of dictionaries, into a dictionary of dictionaries. The first "value" (with key = "collecting institution") of the previous dictionaries is set as the key for each new dictionary, and the values are the rest of the key/value pairs. Also included new keys and values based on previous data (which is very dependant to this precise requeriment)

In [3]:
import json
import os

In [ ]:
with open("/home/pmata/git_repositories/laboratory_address.json") as file:
    orilist = json.load(file)
    final_dict={}
    for x in orilist:
        first_val = list(x.values())[0]
        x["submitting_institution"] = x["collecting_institution"]
        x["submitting_institution_address"] = x["collecting_institution_address"]
        x["submitting_institution_email"] = x["collecting_institution_email"]
        x.pop("collecting_institution")
        final_dict[first_val] = x
print(final_dict)

In [8]:
with open("/home/pmata/git_repositories/relecov-tools/relecov_tools/conf/laboratory_address2.json", "w", encoding="utf-8") as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

Checking if any of the records in the result file after using relecov-tools read-lab-metadata.py has no missing fields

In [5]:
with open("/tmp/relecov/processed_metadata_lab_COD-2129-AND-FIBAO_20230221.json") as file:
    newlist = json.load(file)
    original = len(newlist[0].keys())
    for x in newlist:
        if len(x.keys()) != original:
            print("ERROR")

ERROR


In [246]:
import relecov_tools.utils
from relecov_tools.config_json import ConfigJson

In [36]:
config_json = ConfigJson()
sftp_port = config_json.get_topic_data("sftp_handle", "sftp_port")
print(sftp_port)

22


In [40]:
metadata_tmp_folder = config_json.get_topic_data(
            "sftp_handle", "allowed_R1_delimiters"
                    )
print(metadata_tmp_folder)

['_R1_', '_R1.', '.R1.', 'Forward', 'forward', '_1_', '_1.', '.1.']


---

In [257]:
import json
import os
import relecov_tools.utils

In [248]:
def schema_dicter(schema):
    schema_json = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema}")
    schema_dict = {}
    for key,value in schema_json["properties"].items():
        schema_dict[key] = {x:y for x,y in value.items() if x == "ontology" or x == "label"}
    label_dict = {}
    for x in schema_dict.values():
        for key,value in x.items():
            if key == "label":
                label_dict[value] = x["ontology"]
    return label_dict 

In [ ]:
phagedict = schema_dicter("phage_schema.json")
enadict = schema_dicter("ena_schema.json")
reledict = schema_dicter("relecov_schema.json")
gisaidict = schema_dicter("gisaid_schema.json")
dictlist = {"phage": phagedict, "ena": enadict ,"relecov": reledict, "gisaid":gisaidict}

In [258]:
def schema_differ(schema1, schema2):
    same_ontologies = {k: schema1[k] for k in schema1 if k in schema2 and schema1[k] == schema2[k]}
    diff_ontologies = {k: schema1[k] for k in schema1 if k in schema2 and schema1[k] != schema2[k]}
    
    return same_ontologies, diff_ontologies

In [156]:
print(len(schema_differ(enadict, reledict)[0]), schema_differ(enadict, reledict)[1])
print(len(schema_differ(enadict, phagedict)[0]), schema_differ(enadict, phagedict)[1])
print(len(schema_differ(enadict, gisaidict)[0]), schema_differ(enadict, gisaidict)[1])
print(len(schema_differ(reledict, gisaidict)[0]), schema_differ(reledict, gisaidict)[1])
print(len(schema_differ(reledict, phagedict)[0]), schema_differ(reledict, phagedict)[1])
print(len(schema_differ(gisaidict, phagedict)[0]), schema_differ(gisaidict, phagedict)[1])

#Debugging in-place with import pdb; pdb.set_trace()

# Enviar los errores por correo
# logging.handlers.SMTPHandler(mailhost=("smtp.gmail.com", 465), fromaddr=correo_isciii, toaddrs=correo_usuario, subject="Validation errors", credentials=(usurario,contraseña), secure=None, timeout=1.0)


29 {}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459'}
9 {'Additional Host Information': 'NCIT:C83280'}
14 {}
67 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459', 'Consensus sequence name': 'GENEPIO:0001461'}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123'}


In [ ]:
import pandas as pd
samples_ref = pd.read_csv("/data/bi/research/20230323_TFM_PABLO_MA_RELECOV/ANALYSIS/20230324_ANALYSIS01_AMPLICONS_HUMAN/samples_ref.txt", sep = "\t", header=None)
string_head = "run\tuser\thost\tVirussequence\tsample\ttotalreads\treadshostR1\treadshost\t%readshost\treadsvirus\t%readsvirus\tunmappedreads\t%unmapedreads\tmedianDPcoveragevirus\tCoverage>10x(%)\tVariantsinconsensusx10\tMissenseVariants\t%Ns10x"
print(len(string_head.split("\t")))
samples_list = list(samples_ref[0])
reference_list = list(samples_ref[1])
sample_ref_dict = dict(zip(samples_list, reference_list))
sample_ref_dict

In [253]:
def schema_enumer(schema1, schema2):
    schema_json1 = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema1}")
    schema_json2 = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema2}")
    schema_dict1 = {}
    for key,value in schema_json1["properties"].items():
        if "Enums" in value.keys():
            schema_dict1[key] = [y for x,y in value.items() if x == "Enums"]
    schema_dict2 = {}
    for key,value in schema_json2["properties"].items():
        if "Enums" in value.keys():
            schema_dict2[key] = [y for x,y in value.items() if x == "Enums"]

    enums1 = [x[:-1].split(" [") for k,y in schema_dict1.items() for x in y[0]]
    enums2 = [x[:-1].split(" [") for k,y in schema_dict2.items() for x in y[0]]

    same_enums = {x[0]: [x[1],y[1]] for x in enums1 for y in enums2 if x[0] == y[0] and x[1] == y[1]}
    diff_enums = {x[0]: [x[1],y[1]] for x in enums1 for y in enums2 if x[0] == y[0] and x[1] != y[1]}

    return same_enums, diff_enums


In [254]:
phagedict2 = "phage_schema.json"
enadict2 = "ena_schema.json"
reledict2 = "relecov_schema.json"
gisaidict2 = "gisaid_schema.json"

In [284]:
newdict = {'Research': ['GENEPIO:0100022', 'dfsdf [GENEPIO:0100003'], 'Skin': ['UBERON:0001052', 'sdfsdfd [UBERON:0001003'], 'Middle Nasal Turbinate': ['UBERON:0001762', 'UBERON:0005921'], 'Air filter': ['GENEPIO:0100031', 'ENVO:00003968'], 'Blood Collection Tube': ['GENEPIO:0100031', 'OBI:0002859'], 'Bronchoscope': ['NCIT:C17611', 'OBI:0002826'], 'Collection Container': ['NCIT:C43446', 'OBI:0002088'], 'Filter': ['NCIT:C45801', 'GENEPIO:0100103'], 'Needle': ['NCIT:C69013', 'OBI:0000436'], 'Serum Collection Tube': ['NCIT:C113675', 'OBI:0002860'], 'Sputum Collection Tube': ['GENEPIO:0002115', 'OBI:0002861'], 'Suction Catheter': ['58253008', 'OBI:0002831'], 'Needle Biopsy': ['OBI:0002654', 'OBI:0002651'], 'Filtration': ['NCIT:C16583', 'OBI:0302885'], 'Lavage': ['NCIT:C38068', 'OBI:0600044'], 'Necropsy': ['NCIT:C166270', 'MMO:0000344'], 'Swabbing': ['NCIT:C17627', 'GENEPIO:0002117'], 'Inferior Nasal Turbinate': ['NCIT:C32794', 'UBERON:0005922'], 'Biopsy': ['NCIT:C15189', 'OBI:0002650'], 'Daycare': ['ENVO:01000927', 'GENEPIO:0100193'], 'Hospital': ['ENVO:00002173', 'ECTO:1000035'], 'School': ['ENVO:03501130', 'GENEPIO:0100224']}


In [285]:
fields_to_format = {x: " ".join([f.split(" [", 1)[0] for f in y]) 
            for x,y in newdict.items()}

fields_to_format

{'Research': 'GENEPIO:0100022 dfsdf',
 'Skin': 'UBERON:0001052 sdfsdfd',
 'Middle Nasal Turbinate': 'UBERON:0001762 UBERON:0005921',
 'Air filter': 'GENEPIO:0100031 ENVO:00003968',
 'Blood Collection Tube': 'GENEPIO:0100031 OBI:0002859',
 'Bronchoscope': 'NCIT:C17611 OBI:0002826',
 'Collection Container': 'NCIT:C43446 OBI:0002088',
 'Filter': 'NCIT:C45801 GENEPIO:0100103',
 'Needle': 'NCIT:C69013 OBI:0000436',
 'Serum Collection Tube': 'NCIT:C113675 OBI:0002860',
 'Sputum Collection Tube': 'GENEPIO:0002115 OBI:0002861',
 'Suction Catheter': '58253008 OBI:0002831',
 'Needle Biopsy': 'OBI:0002654 OBI:0002651',
 'Filtration': 'NCIT:C16583 OBI:0302885',
 'Lavage': 'NCIT:C38068 OBI:0600044',
 'Necropsy': 'NCIT:C166270 MMO:0000344',
 'Swabbing': 'NCIT:C17627 GENEPIO:0002117',
 'Inferior Nasal Turbinate': 'NCIT:C32794 UBERON:0005922',
 'Biopsy': 'NCIT:C15189 OBI:0002650',
 'Daycare': 'ENVO:01000927 GENEPIO:0100193',
 'Hospital': 'ENVO:00002173 ECTO:1000035',
 'School': 'ENVO:03501130 GENEPIO:

In [235]:
print(len(schema_enumer(enadict2, reledict2)[0]), schema_enumer(enadict2, reledict2)[1])
print(len(schema_enumer(enadict2, phagedict2)[0]), schema_enumer(enadict2, phagedict2)[1])
print(len(schema_enumer(enadict2, gisaidict2)[0]), schema_enumer(enadict2, gisaidict2)[1])
print(len(schema_enumer(reledict2, gisaidict2)[0]), schema_enumer(reledict2, gisaidict2)[1])
print(len(schema_enumer(reledict2, phagedict2)[0]), schema_enumer(reledict2, phagedict2)[1])
print(len(schema_enumer(gisaidict2, phagedict2)[0]), schema_enumer(gisaidict2, phagedict2)[1])


399 {}
401 {}
20 {}
19 {}
520 {'Research': ['GENEPIO:0100022', 'GENEPIO:0100003'], 'Skin': ['UBERON:0001052', 'UBERON:0001003'], 'Middle Nasal Turbinate': ['UBERON:0001762', 'UBERON:0005921'], 'Air filter': ['GENEPIO:0100031', 'ENVO:00003968'], 'Blood Collection Tube': ['GENEPIO:0100031', 'OBI:0002859'], 'Bronchoscope': ['NCIT:C17611', 'OBI:0002826'], 'Collection Container': ['NCIT:C43446', 'OBI:0002088'], 'Filter': ['NCIT:C45801', 'GENEPIO:0100103'], 'Needle': ['NCIT:C69013', 'OBI:0000436'], 'Serum Collection Tube': ['NCIT:C113675', 'OBI:0002860'], 'Sputum Collection Tube': ['GENEPIO:0002115', 'OBI:0002861'], 'Suction Catheter': ['58253008', 'OBI:0002831'], 'Needle Biopsy': ['OBI:0002654', 'OBI:0002651'], 'Filtration': ['NCIT:C16583', 'OBI:0302885'], 'Lavage': ['NCIT:C38068', 'OBI:0600044'], 'Necropsy': ['NCIT:C166270', 'MMO:0000344'], 'Swabbing': ['NCIT:C17627', 'GENEPIO:0002117'], 'Inferior Nasal Turbinate': ['NCIT:C32794', 'UBERON:0005922'], 'Biopsy': ['NCIT:C15189', 'OBI:0002650']

In [264]:
print(len(schema_enumer(enadict2, gisaidict2)[0]), schema_enumer(enadict2, gisaidict2)[1])


0 {}


In [63]:
with open("/opt/nextstrain/datasets/sars-cov-2-v2/20230511/data/merged_bioinfo_processed_metadata.json") as file:
    merg_bioinfo = json.load(file)
samplesource={}
for x in merg_bioinfo:
    samplesource[x["isolate_sample_id"]] = x["collecting_institution"]


In [70]:
len(list(set(samplesource.values())))

66

In [ ]:

with open("") as file:
    jsonprueba = json.load(file)
jsonclean = [x for x in jsonprueba if "lineage_name" in x.keys()]

with open("/data/bioinfoshare/UCCT_Relecov/COD-2100-MAD-CNM/20220208/bioinfo_processed_metadata_partial.json", "w", encoding="utf-8") as f:
    json.dump(jsonclean, f, ensure_ascii=False, indent=4)

In [37]:
[x["sequence_file_R1_fastq"] for x in jsonclean]

['2018147_R1.fastq.gz',
 '2018187_R1.fastq.gz',
 '2018192_R1.fastq.gz',
 '210010_R1.fastq.gz',
 '210082_R1.fastq.gz',
 '210092_R1.fastq.gz',
 '210098_R1.fastq.gz',
 '210141-B_R1.fastq.gz',
 '210195_R1.fastq.gz',
 '210275_R1.fastq.gz',
 '210359_R1.fastq.gz',
 '212527_R1.fastq.gz']

In [ ]:
annotable = pd.read_csv("/home/pmata/Downloads/GCF_009858895.2_ASM985889v3_genomic.gff", sep="\t", comment="#", header=None)
splitcol = annotable[8].str.split(";", expand=True)

# Drop the original column containing the ";"-separated strings
annotable = annotable.drop(8, axis=1)

# Concatenate the original dataframe with the new columns
annotable = pd.concat([annotable, splitcol], axis=1)
annotable.columns=["Seqid", "Source", "Type", "Start", "End", 
                   "Score", "Strand", "Phase", "ID", "Dbxref", 
                   "collection-date", "country", "gb-acronym", "gbkey", 
                   "genome", "isolate", "mol_type", "nat-host", "old-name"]
annotable

In [ ]:
import re
regex = re.compile(r'(\w+)=([^;]+);?')
annotable = pd.read_csv("/home/pmata/Downloads/GCF_009858895.2_ASM985889v3_genomic.gff", sep="\t", comment="#", header=None)

# Extract the attributes column and split it into multiple columns using the regex
attributes = annotable[8].str.extractall(regex).unstack().droplevel(0, axis=1)
#df.drop(columns=[8], inplace=True)
# Merge the attributes columns with the original DataFrame
df = pd.concat([annotable, attributes], axis=1)

# Remove the original attributes column
attributes

In [127]:
with open("/home/pmata/git_repositories/relecov-tools/relecov_tools/conf/geo_loc_cities.json") as file:
    geolocs = json.load(file)

import pandas as pd
geolocdf = pd.DataFrame.from_dict(geolocs, orient="index"). reset_index()
division = ["city" for x in range(0,len(geolocs))]
geolocdf.insert(loc=0, column="division", value="division")

In [128]:
geolocdf.to_csv("/home/pmata/lat_long2.tsv", sep="\t", index=False, header=False)

In [ ]:
with open("/data/bioinfoshare/UCCT_Relecov/COD-2129-AND-FIBAO/20230331/samples_data_COD-2129-AND-FIBAO_20230331.json") as file:
    samplefile = json.load(file)

submit_id_for_collect_id = {samplefile[x]["sequence_file_R1_fastq"].split(".")[0]:y for x,y in samplefile.items()}
submit_id_for_collect_id

In [83]:
with open("/data/bioinfoshare/UCCT_Relecov/COD-2129-AND-FIBAO/20230331/fixed_samples_data_COD-2129-AND-FIBAO_20230331.json","w", encoding="utf-8") as f:
    json.dump(submit_id_for_collect_id, f, ensure_ascii=False, indent=4)

In [80]:
samplefile["1318725"]["sequence_file_R1_fastq"].split(".")[0]

'AND00132'

In [85]:
import yaml
with open("/data/bioinfoshare/UCCT_Relecov/COD-2129-AND-FIBAO/20230420/software_versions.yml", "r") as file:
    softvers = file.read()
dictvers = yaml.safe_load(softvers)



In [103]:
cleanvers = [x for x in dictvers.values()]
listvers=[]
for dicte in cleanvers:
    for key,value in dicte.items():
        newdict = {key:value}
        if newdict not in listvers:
            listvers.append(newdict)
    if dicte in listvers:
        continue
    else:
        listvers.append(dicte)
listvers2 = [x for x in listvers if len(x) == 1]


In [112]:
versdf = pd.DataFrame(listvers2)
versdf = versdf.melt(var_name="Software", value_name="Version")

In [121]:
versdf = versdf.dropna().reset_index(drop=True)
versdf = versdf.drop(5)

versdf.to_csv("/home/pmata/Downloads/software_versions.csv", sep=",", index=False)

In [125]:
from itertools import groupby

tot = 0
num = 0
with open("/opt/nextstrain/datasets/sars-cov-2-v2/20230511/data/sequences.fasta") as filesamps:
    handle = filesamps.read()
for header, group in groupby(handle, lambda x:x.startswith('>')):
    if not header:
        num += 1
        tot += sum(map(lambda x: len(x.strip()), group))
result = float(tot)/num

In [133]:
mett = pd.read_csv("/opt/nextstrain/datasets/sars-cov-2-v2/20230517/data/example_metadata.tsv", sep="\t")
mett[["country","division"]].drop_duplicates()

,country,division
0,Australia,Victoria
91,China,Beijing
92,China,China
93,China,Wuhan
94,USA,Florida
95,France,France
105,Germany,Bavaria
107,Greece,Athens
117,Guam,Guam
118,China,Hangzhou
